required libraries and APIS's

In [3]:
#installing and downloading
!pip install librosa tensorflow numpy matplotlib seaborn plotly streamlit pyngrok spotipy

dataset downloading and unzipping

In [4]:
import os
!pip install -q kaggle

#kaggle setup
if not os.path.exists('kaggle.json'):
    print("ERROR: Please upload 'kaggle.json' to the Colab file area!")
else:
    #move credentials to the right place
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("Kaggle credentials configured!")

    #downloading dataset
    print("Downloading GTZAN Dataset from Kaggle...")
    !kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

    #unzipping
    print("Unzipping files...")
    !unzip -q gtzan-dataset-music-genre-classification.zip -d /content/genres
    print("Dataset is Ready! Files are in /content/genres/Data/genres_original")

DATASET_PATH="/content/genres/Data/genres_original"

Kaggle credentials configured!
Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
License(s): other
 98% 1.18G/1.21G [00:09<00:00, 172MB/s]
100% 1.21G/1.21G [00:13<00:00, 98.1MB/s]
Unzipping files...
Dataset is Ready! Files are in /content/genres/Data/genres_original


building CNN model and training it

In [5]:
#training CNN
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#configuration
DATASET_PATH="/content/genres/Data/genres_original"
SAMPLE_RATE=22050
DURATION=30
SAMPLES_PER_TRACK=SAMPLE_RATE*DURATION

#preprocessing function
def save_spectrogram(file_path):
    signal,sr=librosa.load(file_path,sr=SAMPLE_RATE)
    start=int(SAMPLES_PER_TRACK/2)-(SAMPLE_RATE*3//2)
    end=start+(SAMPLE_RATE*3)
    signal=signal[start:end]
    mel_spectrogram=librosa.feature.melspectrogram(y=signal,sr=sr,n_mels=128)
    log_mel_spectrogram=librosa.power_to_db(mel_spectrogram)
    if log_mel_spectrogram.shape[1]>128:
        log_mel_spectrogram=log_mel_spectrogram[:,:128]
    else:
        padding=128-log_mel_spectrogram.shape[1]
        log_mel_spectrogram=np.pad(log_mel_spectrogram,((0,0),(0,padding)))
    return log_mel_spectrogram

#preparing data
features=[]
labels=[]
print("Processing audio files... ")
for genre in os.listdir(DATASET_PATH):
    genre_path=os.path.join(DATASET_PATH,genre)
    if os.path.isdir(genre_path):
        print(f"Processing: {genre}")
        for filename in os.listdir(genre_path):
            try:
                data=save_spectrogram(os.path.join(genre_path,filename))
                features.append(data)
                labels.append(genre)
            except: pass

#converting and training
X=np.array(features)[...,np.newaxis]
y_encoded=to_categorical(LabelEncoder().fit_transform(labels))
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2,random_state=42)

model=Sequential([
    Input(shape=(128,128,1)),
    Conv2D(32,(3,3),activation='relu'),MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation='relu'),MaxPooling2D((2,2)),
    Conv2D(128,(3,3),activation='relu'),MaxPooling2D((2,2)),
    Flatten(),Dense(128,activation='relu'),Dropout(0.3),Dense(10,activation='softmax')
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=15,batch_size=32,validation_data=(X_test,y_test))
model.save("music_cnn_model.h5")
np.save("classes.npy",LabelEncoder().fit(labels).classes_)
print("Model Saved!")

Processing audio files... 
Processing: pop
Processing: disco
Processing: jazz


/tmp/ipython-input-3157548450.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  signal,sr=librosa.load(file_path,sr=SAMPLE_RATE)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing: classical
Processing: blues
Processing: rock
Processing: country
Processing: metal
Processing: reggae
Processing: hiphop
Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 216ms/step - accuracy: 0.0855 - loss: 9.7761 - val_accuracy: 0.2600 - val_loss: 2.1552
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.2195 - loss: 2.0595 - val_accuracy: 0.3200 - val_loss: 1.8983
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.3121 - loss: 1.8977 - val_accuracy: 0.4150 - val_loss: 1.6856
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4004 - loss: 1.6490 - val_accuracy: 0.4650 - val_loss: 1.5780
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4893 - loss: 1.4626 - val_accuracy: 0.3700 - val_loss: 1.6520
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5435 - loss: 1.3387 - val_accuracy: 0.5250 - val_loss: 1.3974
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6248 - loss: 1.0802 - val_accuracy: 0.

Model Saved!


importing required libraries, configuration for API's and building the interface of the app

In [16]:
%%writefile app.py
import streamlit as st
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#spotify configuration
CLIENT_ID="paste your client ID"
CLIENT_SECRET="paste your secret key"

#initializing spotify
try:
    sp=spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))
    SPOTIFY_ENABLED=True
except:
    SPOTIFY_ENABLED=False
    print("Spotify Credentials Missing or Invalid")

#page configuration
st.set_page_config(page_title="Vibelytics",page_icon="🎧",layout="wide")

#loading model
@st.cache_resource
def load_classifier():
    model=load_model("music_cnn_model.h5")
    classes=np.load("classes.npy")
    return model,classes

model,classes=load_classifier()

#processing audio
def process_audio(file_path):
    signal,sr=librosa.load(file_path,sr=22050)
    duration_samples=22050*3
    if len(signal)>duration_samples:
        start=int(len(signal)/2)-int(duration_samples/2)
        signal=signal[start:start+duration_samples]
    else:
        padding=duration_samples-len(signal)
        signal=np.pad(signal,(0,padding))
    mel_spectrogram=librosa.feature.melspectrogram(y=signal,sr=sr,n_mels=128)
    log_mel_spectrogram=librosa.power_to_db(mel_spectrogram)
    if log_mel_spectrogram.shape[1]>128: log_mel_spectrogram=log_mel_spectrogram[:,:128]
    else: log_mel_spectrogram=np.pad(log_mel_spectrogram,((0,0),(0,128-log_mel_spectrogram.shape[1])))
    return log_mel_spectrogram,signal,sr

#spotify recommendations
def get_spotify_recs(genre_name):
    if not SPOTIFY_ENABLED: return []
    try:
        #searching for top tracks
        results=sp.search(q=f'genre:{genre_name}',type='track',limit=5)
        tracks=[]
        for item in results['tracks']['items']:
            tracks.append({
                'name': item['name'],
                'artist': item['artists'][0]['name'],
                'image': item['album']['images'][0]['url'],
                'link': item['external_urls']['spotify']
            })
        return tracks
    except Exception as e:
        st.error(f"Spotify API Error: {e}")
        return []

#UI layout
st.title("🎧 Vibelytics 🎧")
st.markdown("Upload an audio file. We will identify the genre and fetch trending hits from Spotify.")
col1,col2=st.columns([1,1])

with col1:
    st.subheader("Upload Music")
    uploaded_file=st.file_uploader("Choose MP3/WAV",type=["mp3","wav"])

    if uploaded_file:
        with open("temp.mp3","wb") as f: f.write(uploaded_file.getbuffer())
        st.audio("temp.mp3")
        if st.button("Analyze Genre",type="primary"):
            with st.spinner("Listening to the texture of the sound..."):
                spec,_,sr=process_audio("temp.mp3")
                prediction=model.predict(spec.reshape(1,128,128,1))
                genre=classes[np.argmax(prediction)]
                confidence=np.max(prediction)

            st.success("Analysis is completed!")
            st.metric("Detected Genre: ",genre.upper())
            st.progress(float(confidence),text=f"Confidence: {confidence*100:.0f}%")

            st.markdown("---")
            st.subheader(f"🔥 Top {genre.title()} on Spotify")

            if SPOTIFY_ENABLED:
                recs=get_spotify_recs(genre)
                if recs:
                    cols=st.columns(5)
                    for i,track in enumerate(recs):
                        with cols[i]:
                            st.image(track['image'])
                            st.caption(f"**{track['name']}**")
                            st.caption(track['artist'])
                            st.markdown(f"[Listen on Spotify 🟢]({track['link']})")
                else:
                    st.warning("No recommendations found for this genre.")
            else:
                st.error("Add Spotify Keys in app.py to see recommendations.")

with col2:
    st.subheader("Spectral Analysis")
    if uploaded_file and 'spec' in locals():
        fig,ax=plt.subplots(figsize=(10,4))
        librosa.display.specshow(spec,sr=sr,x_axis='time',y_axis='mel',cmap='magma',ax=ax)
        ax.set_title('Mel Spectrogram')
        st.pyplot(fig)
        st.info("Vertical lines (beats) and Horizontal lines (tones).")

Overwriting app.py


deploying the app

In [17]:
#running streamlit with Ngrok
from pyngrok import ngrok
import subprocess
import time

#killing previous processes to avoid conflicts
!pkill -f streamlit

NGROK_TOKEN="paste your ngrok token"
ngrok.set_auth_token(NGROK_TOKEN)

#running streamlit in background
process=subprocess.Popen(["streamlit","run","app.py"])
time.sleep(5)

try:
    public_url=ngrok.connect(8501).public_url
    print(f"App is live at: {public_url}")
except Exception as e:
    print(f"Error: {e}")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped!")

App is live at: https://radiochemical-cynthia-extrinsic.ngrok-free.dev
Stopped!
